Author: Akshay Chougule

Created On: Nov 26, 2023

Description: Using OpenAI to create question-answer data based on eligibility criteria of clinical trials (inclusion and exclusion)

In [1]:
# Bring in deps
import os 
import streamlit as st 
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain 
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper 

In [2]:
import sys
sys.path.insert(1, '/home/ubuntu/codebase/my_github/generative-ai-experiments/')
from Constants import OPENAI_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [8]:
eligibility = "Inclusion Criteria: - Has surgically resected and histologically or pathologically confirmed diagnosis of Stage IIB and IIC (pathological or clinical), III, or IV cutaneous melanoma per the American Joint Committee on Cancer (AJCC) eighth edition guidelines -Has not received any prior systemic therapy for melanoma beyond surgical resection - Has had no more than 12 weeks between final surgical resection and randomization - Human immunodeficiency virus (HIV)-infected participants must have well controlled HIV on anti-retroviral therapy (ART) - Participants who are hepatitis B surface antigen (HBsAg) positive are eligible if they have received hepatitis B virus (HBV) antiviral therapy for at least 4 weeks and have undetectable HBV viral load before randomization - Participants with history of hepatitis C virus (HCV) infection are eligible if HCV viral load is undetectable at screening. Exclusion Criteria: - Has ocular, mucosal, or conjunctival melanoma - Has a diagnosis of immunodeficiency or is receiving chronic systemic steroid therapy (in dosing exceeding 10 mg daily of prednisone equivalent) or any other form of immunosuppressive therapy within 7 days prior the first dose of study medication - Has not adequately recovered from major surgical procedure or has ongoing surgical complications - Has received prior radiotherapy within 2 weeks of start of study intervention or has had a history of radiation pneumonitis - Received a live or live attenuated vaccine within 30 days before the first dose of study intervention. Administration of killed vaccines is allowed - Has received an investigational agent or has used an investigational device within 4 weeks before study intervention administration - Has a history of (noninfectious) pneumonitis/interstitial lung disease that required steroids or has current pneumonitis/interstitial lung disease - Has a known additional malignancy that is progressing or has required active treatment within the past 3 years - Has known active central nervous system (CNS) metastases and/or carcinomatous meningitis - Has an active autoimmune disease that has required systemic treatment in past 2 years - Has an active infection requiring systemic therapy - Has had an allogenic tissue/solid organ transplant"

sample_qa = "In the context of description and eligibility of a clinical trial, can patients with history of hepatitis C virus (HCV) infection participate?\nYes, as per the inclusion criteria, the participants with history of hepatitis C virus (HCV) infection are eligible if HCV viral load is undetectable at screening.\n\nIn the context of description and eligibility of a clinical trial, can patients with ocular, mucosal, or conjunctival melanoma participate?\nNo, as per the exclusion criteria if the patient has ocular, mucosal, or conjunctival melanoma, then he or she would not be eligible."

qa_template = PromptTemplate(
    #input_variables = ['description', 'eligibility'], 
    input_variables = ['eligibility','sample_qa'], 
    #template='You are a question-answer pair generator. Could you create 4 sets of question and answers based on this clinical trial eligibility criteria CRITERIA: {title} while leveraging this wikipedia reserch:{wikipedia_research}. And include hashtags to increase the reach'
    template='You are a question-answer pair generator. Could you create 4 sets of question and answers based on this clinical trial eligibility criteria provided in the eligibility input ELIGIBILITY: {eligibility}. Here are two examples of how you can create pairs of question-answers {sample_qa}. Be specific with the question and answers. Avoid broad questions like what is the inclusion criteria. Write each question or answer on a newline.'
)

# Memory 
#title_memory = ConversationBufferMemory(input_key='topic', memory_key='chat_history')

# Llms
llm = OpenAI( temperature=0.5) 
qa_chain = LLMChain(llm=llm, 
                    prompt=qa_template, 
                    verbose=True, 
                    output_key='qa', 
                    #memory=title_memory
                    )


output = qa_chain.run( eligibility=eligibility,sample_qa=sample_qa)



> Entering new LLMChain chain...
Prompt after formatting:
You are a question-answer pair generator. Could you create 4 sets of question and answers based on this clinical trial eligibility criteria provided in the eligibility input ELIGIBILITY: Inclusion Criteria: - Has surgically resected and histologically or pathologically confirmed diagnosis of Stage IIB and IIC (pathological or clinical), III, or IV cutaneous melanoma per the American Joint Committee on Cancer (AJCC) eighth edition guidelines -Has not received any prior systemic therapy for melanoma beyond surgical resection - Has had no more than 12 weeks between final surgical resection and randomization - Human immunodeficiency virus (HIV)-infected participants must have well controlled HIV on anti-retroviral therapy (ART) - Participants who are hepatitis B surface antigen (HBsAg) positive are eligible if they have received hepatitis B virus (HBV) antiviral therapy for at least 4 weeks and have undetectable HBV viral load bef

### The cost was 2 cents to execute this with OpenAI.

In [10]:
print(output)



In the context of description and eligibility of a clinical trial, can participants who are hepatitis B surface antigen (HBsAg) positive participate?
Yes, as per the inclusion criteria, participants who are hepatitis B surface antigen (HBsAg) positive are eligible if they have received hepatitis B virus (HBV) antiviral therapy for at least 4 weeks and have undetectable HBV viral load before randomization.

In the context of description and eligibility of a clinical trial, can patients who have received prior radiotherapy within 2 weeks of start of study intervention participate?
No, as per the exclusion criteria if the patient has received prior radiotherapy within 2 weeks of start of study intervention, then he or she would not be eligible.


In [11]:
output

'\n\nIn the context of description and eligibility of a clinical trial, can participants who are hepatitis B surface antigen (HBsAg) positive participate?\nYes, as per the inclusion criteria, participants who are hepatitis B surface antigen (HBsAg) positive are eligible if they have received hepatitis B virus (HBV) antiviral therapy for at least 4 weeks and have undetectable HBV viral load before randomization.\n\nIn the context of description and eligibility of a clinical trial, can patients who have received prior radiotherapy within 2 weeks of start of study intervention participate?\nNo, as per the exclusion criteria if the patient has received prior radiotherapy within 2 weeks of start of study intervention, then he or she would not be eligible.'

## Trial 2
Refining prompt to generate more questions

In [12]:
eligibility = "Inclusion Criteria: - Has surgically resected and histologically or pathologically confirmed diagnosis of Stage IIB and IIC (pathological or clinical), III, or IV cutaneous melanoma per the American Joint Committee on Cancer (AJCC) eighth edition guidelines -Has not received any prior systemic therapy for melanoma beyond surgical resection - Has had no more than 12 weeks between final surgical resection and randomization - Human immunodeficiency virus (HIV)-infected participants must have well controlled HIV on anti-retroviral therapy (ART) - Participants who are hepatitis B surface antigen (HBsAg) positive are eligible if they have received hepatitis B virus (HBV) antiviral therapy for at least 4 weeks and have undetectable HBV viral load before randomization - Participants with history of hepatitis C virus (HCV) infection are eligible if HCV viral load is undetectable at screening. Exclusion Criteria: - Has ocular, mucosal, or conjunctival melanoma - Has a diagnosis of immunodeficiency or is receiving chronic systemic steroid therapy (in dosing exceeding 10 mg daily of prednisone equivalent) or any other form of immunosuppressive therapy within 7 days prior the first dose of study medication - Has not adequately recovered from major surgical procedure or has ongoing surgical complications - Has received prior radiotherapy within 2 weeks of start of study intervention or has had a history of radiation pneumonitis - Received a live or live attenuated vaccine within 30 days before the first dose of study intervention. Administration of killed vaccines is allowed - Has received an investigational agent or has used an investigational device within 4 weeks before study intervention administration - Has a history of (noninfectious) pneumonitis/interstitial lung disease that required steroids or has current pneumonitis/interstitial lung disease - Has a known additional malignancy that is progressing or has required active treatment within the past 3 years - Has known active central nervous system (CNS) metastases and/or carcinomatous meningitis - Has an active autoimmune disease that has required systemic treatment in past 2 years - Has an active infection requiring systemic therapy - Has had an allogenic tissue/solid organ transplant"

sample_qa = "In the context of description and eligibility of a clinical trial, can patients with history of hepatitis C virus (HCV) infection participate?\nYes, as per the inclusion criteria, the participants with history of hepatitis C virus (HCV) infection are eligible if HCV viral load is undetectable at screening.\n\nIn the context of description and eligibility of a clinical trial, can patients with ocular, mucosal, or conjunctival melanoma participate?\nNo, as per the exclusion criteria if the patient has ocular, mucosal, or conjunctival melanoma, then he or she would not be eligible."

qa_template = PromptTemplate(
    #input_variables = ['description', 'eligibility'], 
    input_variables = ['eligibility','sample_qa'], 
    #template='You are a question-answer pair generator. Could you create 4 sets of question and answers based on this clinical trial eligibility criteria CRITERIA: {title} while leveraging this wikipedia reserch:{wikipedia_research}. And include hashtags to increase the reach'
    template='You are a question-answer pair generator. Could you create 8 sets of question and answers based on this clinical trial eligibility criteria provided in the eligibility input ELIGIBILITY: {eligibility}. Here are two examples of how you can create pairs of question-answers {sample_qa}. Be specific with the question and answers. Avoid broad questions like what is the inclusion criteria. Write each question or answer on a newline.'
)

# Memory 
#title_memory = ConversationBufferMemory(input_key='topic', memory_key='chat_history')

# Llms
llm = OpenAI( temperature=0.5) 
qa_chain = LLMChain(llm=llm, 
                    prompt=qa_template, 
                    verbose=True, 
                    output_key='qa', 
                    #memory=title_memory
                    )


output = qa_chain.run( eligibility=eligibility,sample_qa=sample_qa)

print(output)



> Entering new LLMChain chain...
Prompt after formatting:
You are a question-answer pair generator. Could you create 8 sets of question and answers based on this clinical trial eligibility criteria provided in the eligibility input ELIGIBILITY: Inclusion Criteria: - Has surgically resected and histologically or pathologically confirmed diagnosis of Stage IIB and IIC (pathological or clinical), III, or IV cutaneous melanoma per the American Joint Committee on Cancer (AJCC) eighth edition guidelines -Has not received any prior systemic therapy for melanoma beyond surgical resection - Has had no more than 12 weeks between final surgical resection and randomization - Human immunodeficiency virus (HIV)-infected participants must have well controlled HIV on anti-retroviral therapy (ART) - Participants who are hepatitis B surface antigen (HBsAg) positive are eligible if they have received hepatitis B virus (HBV) antiviral therapy for at least 4 weeks and have undetectable HBV viral load bef